In [195]:
import menzalib as mz
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.constants import c

In [207]:
def theta(x):
    if x>0: return 1
    return 0

def f2t(V,V0,a,b,I0):
    return a*theta(V0-V)*(V0-V)**2+b*V+I0
f2=np.vectorize(f2t)

def cifrat(x):
    i=0
    while (x-np.round(x,-i))!=0: i=i-1
    return i
cifra=np.vectorize(cifrat)

def lin(x,a,b):
    return a*x+b


# Fit con la funzione con esponente variabile
Sto usando la funzione `f2(V,V0,a,b,I0,2)`$=a(V_0-V)^2\theta(V_0-V)+bV+I_0$

Le immagini sono salvate nella cartella `immagini/` e le tabelle per latex nella cartella `tabelle/`

In [210]:
lung=[450,499,545,577]
v0=[1.213,0.951,0.774,0.715]
dv0=np.zeros(len(v0))
p0=np.zeros((len(lung),4))
for i in range(len(lung)):
    #raccolta dati e calcolo deli errori
    V,I=np.genfromtxt('dati/'+str(lung[i])+'nm.txt',unpack='True')
    I=I*1e-12
    V=V*1e-3
    dV=mz.dVdig(V)
    dI=np.zeros(len(I))
    for j in range(len(I)):
        dI[j]=np.sqrt((I[j]*4e-3)**2+10.0**(cifra(I[j])*2))
        if I[j]==0: dI[j]=1e-13
    p0[i][0]=v0[i]
    
    #il fit con gli errori sulle x non converge per i dati a 577nm
    popt,pcov=mz.curve_fitdx(f2,V,I,dV,dI,p0[i])
    chi2,pval=mz.chi2_pval(f2,V,I,dI,popt,dV)
    v0[i]=popt[0]
    dv0[i]=pcov[0][0]
    
    #plottaggio
    plt.figure()
    plt.subplot(211)
    plt.errorbar(V,-I,xerr=dV,yerr=dI,fmt='.')
    plt.xlabel('Voltaggio[V]')
    plt.ylabel('Corrente[A]')
    x=np.linspace(V[0],V[-1],100)
    y=f2(x,*popt)
    plt.plot(x,-y)
    
    plt.subplots_adjust(hspace=0.5)
    
    plt.subplot(212)
    plt.errorbar(V,-(I-f2(V,*popt)),xerr=dV,yerr=dI,fmt='.')
    plt.xlabel('Voltaggio[V]')
    plt.ylabel('Residui della corrente[A]')
    x=np.linspace(V[0],V[-1],100)
    y=np.zeros(len(x))
    plt.plot(x,y)
    plt.savefig('immagini/'+str(lung[i])+'nm.eps',format='eps')
    plt.close()
    
    #faccio la tabella con i parametri di fit
    col1=['$V_0$','$a$','$b$','$I_0$','$\chi^2$']
    col2=mz.ne_tex(popt,pcov.diagonal())
    col2=np.append(col2,mz.ns_tex(chi2))
    mz.mat_tex([col1,col2],'Parametri di Fit & Valori di Fit','tabelle/'+str(lung[i])+'nm.tex')